In [1]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage      
import sklearn.feature_extraction.text as sk_text
import sklearn.cluster as sk_cluster
import sklearn.metrics as metrics
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', -1)
import datetime
from sklearn import preprocessing


# Remove all retweets first. Remove all users that have less than 20 tweets. Use train_test_split() to split data into training and test sets, where 20 percent of the records go to test set.

In [2]:
df_cols = ['Name', 'ScreenName', 'UserID', 'FollowersCount', 'FriendsCount', 'Location', 'Description', 'CreatedAt', 'StatusID', 'Language', 'Place', 'RetweetCount', 'FavoriteCount','Text']
df = pd.read_table("clinton_trump_tweets.txt",encoding="ISO-8859-1",header=None,names=df_cols)
df_cols = ['User_id', 'Choice']
df1 = pd.read_table("clinton_trump_user_classes.txt",encoding="ISO-8859-1",header=None,names=df_cols)

In [3]:
users20 = df.groupby('UserID')['Text'].count().sort_values(ascending=False)

In [4]:
users20_list = users20[users20<20].index.tolist()

In [5]:
ucheck = pd.Series(df.UserID)
newdf=df.drop(df[ucheck.isin(users20_list)].index)

In [6]:

users20check = newdf.groupby('UserID')['Text'].count().sort_values(ascending=False)

In [7]:
def dropRT(my_list):
    return my_list[0]=="RT "

In [8]:
df.drop(df[df.Text[:].str.split('@').apply(dropRT)].index,inplace=True)

In [9]:
df

,Name,ScreenName,UserID,FollowersCount,FriendsCount,Location,Description,CreatedAt,StatusID,Language,Place,RetweetCount,FavoriteCount,Text
0,Cebel,Cebel6,1519696717,132,263,"Little Rock, Arkansas",Arkansas Razorback Fan Just trying to be #Uncommon one 1-0 day at a time.,Sat Oct 29 08:10:06 EEST 2016,792232017094119425,en,NaN,0,1,@NWAJimmy I've read it now though brother. Was pretty spot on Lots of bright spots but a lot to work on. Exactly as an exhibition should be!
2,nolaguy,nolaguy_phd,1450086582,797,1188,NaN,"An LSU Ph.D student living in New Orleans, trying to find a second act.",Sat Oct 29 21:53:29 EEST 2016,792439227090767872,en,NaN,0,0,@gaystoner821 I think New Orleans spoiled me with food. I need to try and branch out in BR.
6,David Walling,davidjwalling,106568768,975,2781,"Dallas, TX",Bloodletting secure algorithms close to the bone. #HealthIT security matters. Opinions my own. https://t.co/Dcpe6FteOq,Sat Oct 29 00:16:48 EEST 2016,792112907488079872,en,NaN,0,0,#infosec #Intel #ACM #IEEE Impacts Haswell microarch. Paper proposes mitigations that could prevent BTB-based side https://t.co/DW6vgRAPrv
7,robert2266,robert2266,17101060,845,938,The Universe,The Dark Lord,Fri Oct 28 14:41:06 EEST 2016,791968028191711237,en,NaN,0,0,"Hacked e-mails show Clinton campaigns fears about Sanders | https://t.co/WMyCHuCDIc The Philippine Star (PhilippineStar) October 28, 2"
10,neddyo,neddyo,16818809,1400,379,Long Island and beyond...,"You should be digging it while it's happening - Zappa; I don't know how you do it, man. - Anastasio #minimix #recommNeds #NYCSOTW",Mon Oct 31 08:06:52 EET 2016,792971077836124160,en,NaN,0,1,Hulk smash!
12,Alexandria Taylor,twentythreeofme,163712593,1342,1346,"Portland, Oregon",Activist | Creator | Educator | #FemTube Host | Passion Project: #DisabilitiesOnYT Panels | Want to participate? https://t.co/IYy6JVqmo5,Tue Oct 25 19:41:52 EEST 2016,790956555030061056,en,"PlaceJSONImpl{name='Washington', streetAddress='null', countryCode='US', id='bc3a38d3d5999b4b', country='United States', placeType='admin', url='https://api.twitter.com/1.1/geo/id/bc3a38d3d5999b4b.json', fullName='Washington, USA', boundingBoxType='Polygon', boundingBoxCoordinates=[[Ltwitter4j.GeoLocation;@5ad5215f], geometryType='null', geometryCoordinates=null, containedWithIn=[]}",0,7,"Well, I'm glad you got a good haircut, America's in the toilet. Is this not a conversation @Haylie_Bre and I woul https://t.co/6dVIIKZcgv"
13,Harry Binford,CaptainNormal,2447279666,895,1224,USA,"Proud Christian husband, frequent shooting range attendee, and former law enforcement. Lifetime member of the NRA. Pro-life, Jesus is Lord. America first.",Thu Oct 27 21:44:44 EEST 2016,791712249685676032,en,NaN,0,0,@WayneDupreeShow #climatechange is the democrats' attempt to play God
18,Matthew Carl Barnard,matthewcbarnard,231921777,350,668,"Milwaukee, Wisconsin",Trying to put one foot forward each day towards living my dream. I have thoughts along the way. Enjoy.,Wed Oct 26 04:23:47 EEST 2016,791087900368789504,en,NaN,0,0,OH MY GOD. TEARS. https://t.co/8odjnHZwRK
20,mystikal knight,_Poochivicious,235657662,777,3236,On An Ultra Light Beams,my life is currently a master p sneaker,Fri Oct 28 15:04:44 EEST 2016,791973973106057216,en,NaN,0,0,"no lie, I been waiting for tony's story https://t.co/S5i4ESNilb"
21,Jim Cook,valukind,201499452,1413,620,New York City,"Internet marketer, college grad, Air Force Vet.Capt.,like people, cats, football,fine dining and helping others to achieve.",Fri Oct 28 03:22:20 EEST 2016,791797209335418880,en,NaN,0,0,How a dot-com millionaire became a social media crusader against solar amendment: It was a violation of Googl... https://t.co/ychypPZpKd


In [10]:
from sklearn.model_selection import train_test_split

In [11]:
df_hashtag_agg  = df.groupby('UserID')['Text'].sum()
df_ready_for_sklearn = pd.DataFrame({'User_id':df_hashtag_agg.index, 'Tweets': df_hashtag_agg.values})

In [12]:
vectorizer = sk_text.TfidfVectorizer(max_features = 5000, min_df = 100, stop_words = 'english')

In [13]:
mergedText = pd.merge(df_ready_for_sklearn, df1, on='User_id')

In [14]:
mergedText

User_id  \
0       150          
1       1437         
2       1501         
3       1512         
4       1644         
5       1668         
6       1737         
7       2294         
8       2311         
9       2391         
10      2424         
11      2426         
12      2480         
13      10420        
14      10549        
15      10637        
16      10715        
17      10933        
18      11014        
19      11569        
20      11862        
21      12404        
22      13512        
23      13558        
24      13683        
25      13737        
26      14763        
27      15543        
28      15833        
29      16373        
...       ...        
164776  2521842644   
164777  2521889909   
164778  2521921878   
164779  2521925561   
164780  2521929295   
164781  2521930375   
164782  2521930666   
164783  2521933298   
164784  2521935452   
164785  2521952112   
164786  2521964308   
164787  2521967087   
164788  2521976250   
164789  2521976948   
164790  2522077191   
164791  2522160205   
164792  2522174788   
164793  2522199981   
164794  2522209999   
164795  2522217890   
164796  2522238208   
164797  2522313516   
164798  2522333254   
164799  2522444346   
164800  2522473626   
164801  2522477492   
164802  2522544348   
164803  2522594107   
164804  2522622542   
164805  2522653212   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [15]:
matrix = vectorizer.fit_transform(mergedText.Tweets.values)

In [16]:
tfidf = matrix.toarray()

In [17]:
tfidf=tfidf[0:5000,:]

In [18]:
y = mergedText['Choice'][0:5000]

In [19]:
y.shape

(5000,)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(tfidf, y, test_size=0.2, random_state=4)

In [23]:
X_train.shape

(8000, 10000)

In [24]:
X_test.shape

(2000, 10000)

In [25]:
y_train.shape

(8000,)

In [26]:
y_test.shape

(2000,)

# Task 1.2 (20 pts): Train Decision Tree, SVM, Logistic Regression, and Neural Networks.

## Decision Tree

In [41]:
from sklearn import tree

dtree = tree.DecisionTreeClassifier()

dtree = dtree.fit(X_train, y_train)

y_pred = dtree.predict(X_test)

print("test", y_test)
print("pred", y_pred)    

test 13249    1
8335     0
13075    1
2764     1
279      1
6713     1
2319     1
1959     1
702      0
8052     1
5099     1
2992     1
1448     0
11480    1
14906    1
2454     1
6320     1
1374     1
14897    1
7562     1
1445     0
6250     0
10021    1
7672     1
6684     1
11962    1
4669     1
4054     0
13274    1
14990    1
        ..
1397     0
8920     1
6879     0
9477     1
12133    1
2589     1
14926    1
11144    1
3787     1
10812    0
6205     1
11424    1
6462     1
5622     1
14928    1
87       1
2981     1
13376    1
689      0
8900     1
13543    1
1009     0
12620    1
1079     0
4815     1
9017     1
11110    1
9684     1
115      1
8731     1
Name: Choice, Length: 3000, dtype: int64
pred [1 1 1 ... 1 0 1]


In [42]:
print(metrics.confusion_matrix(y_test,y_pred))

print(metrics.precision_score(y_test,y_pred, average='weighted'))   # Parameter "average" is requred if not a binary model
print(metrics.recall_score(y_test,y_pred, average='weighted'))      # Parameter "average" is requred if not a binary model
print(metrics.f1_score(y_test,y_pred, average='weighted'))          # Parameter "average" is requred if not a binary model

[[ 195  504]
 [ 425 1876]]
0.6778587286527514
0.6903333333333334
0.6836728487510768


In [43]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.31      0.28      0.30       699
           1       0.79      0.82      0.80      2301

   micro avg       0.69      0.69      0.69      3000
   macro avg       0.55      0.55      0.55      3000
weighted avg       0.68      0.69      0.68      3000




## SVM 

In [44]:
from sklearn import svm

svm_clf = svm.SVC(gamma='auto')
svm_clf.fit(X_train,y_train)


y_pred = svm_clf.predict(X_test)

print("test", y_test[:10])
print("pred", y_pred[:10])    

test 13249    1
8335     0
13075    1
2764     1
279      1
6713     1
2319     1
1959     1
702      0
8052     1
Name: Choice, dtype: int64
pred [1 1 1 1 1 1 1 1 1 1]


In [45]:
print(metrics.confusion_matrix(y_test,y_pred))

print(metrics.precision_score(y_test,y_pred, average='weighted'))   # Parameter "average" is requred if not a binary model
print(metrics.recall_score(y_test,y_pred, average='weighted'))      # Parameter "average" is requred if not a binary model
print(metrics.f1_score(y_test,y_pred, average='weighted'))          # Parameter "average" is requred if not a binary model

[[   0  699]
 [   0 2301]]
0.588289
0.767
0.6658619128466328


/Users/muayad/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/muayad/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [46]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       699
           1       0.77      1.00      0.87      2301

   micro avg       0.77      0.77      0.77      3000
   macro avg       0.38      0.50      0.43      3000
weighted avg       0.59      0.77      0.67      3000



/Users/muayad/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Neural Networks 

In [47]:
from sklearn.preprocessing import StandardScaler

In [48]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_nn = scaler.transform(X_train)
X_test_nn = scaler.transform(X_test)

In [49]:
from sklearn.neural_network import MLPClassifier

In [50]:
#hidden_layer_sizes: 3 hidden layers,  each has 30 neurons

# note that max_iter determines the number of epochs
mlp = MLPClassifier(hidden_layer_sizes=(30,30,30), solver='adam', max_iter=1000)
mlp.fit(X_train_nn, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(30, 30, 30), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [51]:
y_pred = mlp.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [52]:
print(metrics.confusion_matrix(y_test,y_pred))

print(metrics.precision_score(y_test,y_pred, average='weighted'))   # Parameter "average" is requred if not a binary model
print(metrics.recall_score(y_test,y_pred, average='weighted'))      # Parameter "average" is requred if not a binary model
print(metrics.f1_score(y_test,y_pred, average='weighted'))          # Parameter "average" is requred if not a binary model

[[   0  699]
 [   0 2301]]
0.588289
0.767
0.6658619128466328


/Users/muayad/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/muayad/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Logistic Regression

In [53]:
import sklearn.linear_model as linear_model

lr_clf = linear_model.LogisticRegression(solver='liblinear', multi_class='ovr')
lr_clf.fit(X_train, y_train)

y_pred = lr_clf.predict(X_test)

print("test", y_test)
print("pred", y_pred)    

test 13249    1
8335     0
13075    1
2764     1
279      1
6713     1
2319     1
1959     1
702      0
8052     1
5099     1
2992     1
1448     0
11480    1
14906    1
2454     1
6320     1
1374     1
14897    1
7562     1
1445     0
6250     0
10021    1
7672     1
6684     1
11962    1
4669     1
4054     0
13274    1
14990    1
        ..
1397     0
8920     1
6879     0
9477     1
12133    1
2589     1
14926    1
11144    1
3787     1
10812    0
6205     1
11424    1
6462     1
5622     1
14928    1
87       1
2981     1
13376    1
689      0
8900     1
13543    1
1009     0
12620    1
1079     0
4815     1
9017     1
11110    1
9684     1
115      1
8731     1
Name: Choice, Length: 3000, dtype: int64
pred [1 1 1 ... 1 1 1]


In [54]:
print(metrics.confusion_matrix(y_test,y_pred))
print(metrics.precision_score(y_test,y_pred, average='weighted'))   # Parameter "average" is requred if not a binary model
print(metrics.recall_score(y_test,y_pred, average='weighted'))      # Parameter "average" is requred if not a binary model
print(metrics.f1_score(y_test,y_pred, average='weighted'))          # Parameter "average" is requred if not a binary model

[[  54  645]
 [  21 2280]]
0.7656266666666667
0.778
0.7017653592502603


In [55]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.08      0.14       699
           1       0.78      0.99      0.87      2301

   micro avg       0.78      0.78      0.78      3000
   macro avg       0.75      0.53      0.51      3000
weighted avg       0.77      0.78      0.70      3000



# Task 1.3 (20 pts): Train k-NN model.

In [21]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(X_train,y_train)

y_pred = knn.predict(X_test)



### Apply 5-fold cross validation and Perform parameter tuning on k-NN model.

In [22]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(knn, tfidf, y, cv=5, scoring='f1_weighted')

In [ ]:
k_range = list(range(10, 35)) #have to figure out what k_range is
k_scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, tfidf, y, cv=5, scoring='f1_weighted')
    k_scores.append(scores.mean())
print(k_scores)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# plot the value of K for KNN (x-axis) versus the cross-validated accuracy (y-axis)
plt.plot(k_range, k_scores) 
plt.xlabel('Value of K for KNN')
plt.ylabel('F1 Score based on Cross-Validation')
plt.show()

## Apply 5-fold cross validation and use grid search to find the best K value for k-NN model


In [ ]:
k_range = list(range(10, 35))
param_grid = dict(n_neighbors=k_range)

knn = KNeighborsClassifier(n_neighbors=1)

In [ ]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(knn, param_grid, cv=5, scoring='f1_weighted')

In [ ]:
grid.fit(tfidf, y)

In [ ]:
means = grid.cv_results_['mean_test_score']
means

In [ ]:
grid.cv_results_['params']

In [ ]:
for mean, params in zip(means, grid.cv_results_['params']):
    print(mean, params)

In [ ]:
# plot the results
plt.plot(k_range, means)
plt.xlabel('Value of K for KNN')
plt.ylabel('F1 score based on Cross-Validation')
plt.show()

In [ ]:
# identify the best model

print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

In [ ]:
print(metrics.confusion_matrix(y_test,y_pred))
print(metrics.precision_score(y_test,y_pred, average='weighted'))   # Parameter "average" is requred if not a binary model
print(metrics.recall_score(y_test,y_pred, average='weighted'))      # Parameter "average" is requred if not a binary model
print(metrics.f1_score(y_test,y_pred, average='weighted'))          # Parameter "average" is requred if not a binary model

In [ ]:
print(metrics.classification_report(y_test, y_pred))